In [1]:
!pip install selenium

You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import os
import urllib

# Scrape with selenium
linked in is a dynamic site therefor seleniumis the best option, BeautifulSoup didn't work
https://chromedriver.chromium.org/downloads -> download the newest version

In [4]:
# Statistisches Bundesamt - Großstädte Deutschland
# https://de.statista.com/statistik/daten/studie/1353/umfrage/einwohnerzahlen-der-grossstaedte-deutschlands/
regions = ["Berlin", "Hamburg", "München", "Köln", "Frankfurt am Main", "Stuttgart", "Düsseldorf", "Leipzig", "Dortmund", "Essen", "Bremen", "Dresden", "Hannover", "Nürnberg", "Duisburg", "Bochum", "Wuppertal", "Bielefeld", "Bonn", "Münster", "Mannheim", "Karlsruhe", "Augsburg", "Wiesbaden", "Mönchengladbach", "Gelsenkirchen", "Aachen", "Braunschweig", "Kiel", "Chemnitz",
           "Halle (Saale)", "Magdeburg", "Freiburg im Breisgau", "Krefeld", "Mainz", "Lübeck", "Erfurt", "Oberhausen", "Rostock", "Kassel", "Hagen", "Potsdam", "Saarbrücken", "Hamm", "Ludwigshafen am Rhein", "Mülheim a. d. Ruhr", "Oldenburg", "Osnabrück", "Leverkusen", "Darmstadt", "Heidelberg", "Solingen", "Herne", "Regensburg", "Neuss", "Paderborn", "Ingolstadt", "Offenbach am Main", "Fürth", "Ulm", "Würzburg", "Heilbronn", "Pforzheim", "Wolfsburg", "Bottrop", "Göttingen", "Reutlingen", "Koblenz", "Erlangen", "Bremerhaven", "Remscheid", "Bergisch Gladbach", "Recklinghausen", "Trier", "Jena", "Moers", "Salzgitter", "Siegen", "Gütersloh", "Hildesheim", "Kaiserslautern"]



In [ ]:
# "Data Scientist" alternative runs:
# Data Analyst
# Machine Learning Expert
# Big Data Engineer

In [5]:
st = time.time()
list = []
for i in range(0, len(regions)):
    URL = "https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=" + \
        regions[i] + "%2C%20DE"
    wd = webdriver.Chrome('./chromedriver.exe')
    wd.get(URL)

    print(i, regions[i], "fetching...")
    # test
    # resultCount = wd.find_element(By.CLASS_NAME,'results-context-header__job-count')
    # resultCount.get_attribute('innerText')
    # postings = wd.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
    # len(postings)

    # the list is an endless scroll therefore scroll to the bottom a few times with js
    # the list is limited to 1000 results, 25 per 'page', 1000/25 = 40 scrolls but 50 to be save at the botttom
    for j in range(50):
        try:
            btn = wd.find_element(
                By.CSS_SELECTOR, "button.infinite-scroller__show-more-button")
        # print(btn.is_displayed())
        # optimization: stop fetching if scrolling is not possible: see-more-jobs__viewed-all is visible
            if btn.is_displayed():
                btn.click()
            else:
                wd.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")
                try:
                    finished = wd.find_element(
                        By.CSS_SELECTOR, "div.see-more-jobs__viewed-all")
                    if finished.is_displayed():
                        break
                except:
                    continue
        except:
            break
        # there is a limit for no account requests which will trigger if we scroll too fast
        time.sleep(2)

    # confirm the list is longer than initial 25, every scroll loads 25, should be 1000
    postings = wd.find_elements(
        By.CSS_SELECTOR, "ul.jobs-search__results-list li")

    print(i, regions[i], len(postings), "posts")

    indexList = []
    indexErrors = []
    for j in range(len(postings)):
        link = ''
        title = ''
        company = ''
        location = ''
        listdate = ''
        try:
            post = postings[j]
            title = post.find_element(
                By.CSS_SELECTOR, ".base-search-card__title").get_attribute('innerText')
            company = post.find_element(
                By.CSS_SELECTOR, ".base-search-card__subtitle").get_attribute('innerText')
            location = post.find_element(
                By.CSS_SELECTOR, ".job-search-card__location").get_attribute('innerText')
            listdate = post.find_element(
                By.CSS_SELECTOR, "time").get_attribute('datetime')
            try:
                link = post.find_element(
                    By.CSS_SELECTOR, "a.base-card__full-link").get_attribute('href')
            except:
                link = post.find_element(
                    By.CSS_SELECTOR, "a").get_attribute('href')
        except:
            indexErrors.append(j)

        indexList.append({
            "link": link or '',
            "title": title or '',
            "company": company or '',
            "location": location or '',
            "listdate": listdate or '',
            "city": regions[i]
        })
    list.append({"cityIndex": i, "jobs": indexList, "errors": indexErrors})
    print(i, regions[i], len(indexList), "jobs")
    print(i, regions[i], len(indexErrors), "errors")
    print(i, regions[i], "finished")
    wd.close()  # close after each page
print("finished fetching jobs")
finalList = []
for city in list:
    finalList.extend(city["jobs"])

print(len(finalList), "jobs fetched")

with open("data_analyst_cities.json", "w") as outfile:
    json.dump(finalList, outfile, indent=4, sort_keys=False)

print("saved to json")

et = time.time()
elapsed_time = et - st
print(elapsed_time)
# os.system("shutdown /s /t 1")


C:\Users\denni\AppData\Local\Temp/ipykernel_13256/1914152496.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('./chromedriver.exe')


0 Berlin fetching...


KeyboardInterrupt: 

# data cleanup

In [4]:
df = pd.read_json('data_scientist_cities.json')
df.head()

,link,title,company,location,listdate,city
0,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist,Prestatech,Berlin,2022-12-07,Berlin
1,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (F/M/X),omnius,Berlin,2023-01-20,Berlin
2,https://de.linkedin.com/jobs/view/machine-lear...,Machine Learning Engineer (Berlin),Enjins,Berlin,2022-12-12,Berlin
3,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (f/m/d),Aignostics,Berlin,2023-01-05,Berlin
4,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (f/m/d),Codepan GmbH,Berlin,2023-01-16,Berlin


In [5]:
df.shape[0]

62844

In [47]:
print(df.iloc[3].link)

https://de.linkedin.com/jobs/view/data-scientist-f-m-d-at-aignostics-3421358993?refId=QiBE3u0AoU4TDGUEb2o1lA%3D%3D&trackingId=u0AzK46gmNND%2BJU6erysKg%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card


In [52]:
def remove_query(full_url):
    r = urllib.parse.urlparse(full_url)
    link = urllib.parse.urlunparse((r.scheme, r.netloc, r.path,'','',''))
    return link

In [53]:
remove_query(df.iloc[3].link)

'https://de.linkedin.com/jobs/view/data-scientist-f-m-d-at-aignostics-3421358993'

In [54]:
df['link'] = df['link'].apply(remove_query)

In [61]:
print(df.iloc[6].link)

https://de.linkedin.com/jobs/view/data-scientist-media-m-f-d-at-lunar-x-3451894732


In [62]:
df2 = df.drop_duplicates(subset=['link'])

In [63]:
df2.shape

(27777, 6)

In [64]:
print(df2.iloc[6].link)

https://de.linkedin.com/jobs/view/data-scientist-media-m-f-d-at-lunar-x-3451894732


In [65]:
df2.to_csv("cleaned_unique.csv",index=False)

# scrape full text

## initial run

In [30]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import os
import urllib

In [8]:
df = pd.read_csv('cleaned_unique.csv')
df = df.assign(content=None)
df.head()

,link,title,company,location,listdate,city,content
0,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist,Prestatech,Berlin,2022-12-07,Berlin,None
1,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (F/M/X),omnius,Berlin,2023-01-20,Berlin,None
2,https://de.linkedin.com/jobs/view/machine-lear...,Machine Learning Engineer (Berlin),Enjins,Berlin,2022-12-12,Berlin,None
3,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (f/m/d),Aignostics,Berlin,2023-01-05,Berlin,None
4,https://de.linkedin.com/jobs/view/data-scienti...,Data Scientist (f/m/d),Codepan GmbH,Berlin,2023-01-16,Berlin,None


In [12]:
df.shape

(27777, 7)

In [5]:
options = webdriver.ChromeOptions() 
# options.headless = True 
wd = webdriver.Chrome('./chromedriver.exe', options=options)

C:\Users\denni\AppData\Local\Temp/ipykernel_12768/4189031768.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('./chromedriver.exe', options=options)


In [14]:
errors = []
for index, row in df.iterrows():
    # print(index, "/", df.shape[0])
    try:
        wd.get(row.link)
        content = wd.find_element(
            By.CSS_SELECTOR, "div.show-more-less-html__markup")
        df.loc[index, 'content'] = content.get_attribute("innerText")
        print(index, "/", df.shape[0],":",len(df.loc[index, 'content']))

    except:
        df.loc[index, 'content'] = ""
        errors.append(index)
        print(index, "/", df.shape[0],": error")

    time.sleep(5)
df.to_csv("unique_jobs_with_content.csv",index=False)

0 / 27777 : 2953
1 / 27777 : 3206
2 / 27777 : 4739
3 / 27777 : 2754
4 / 27777 : 2776
5 / 27777 : 3732
6 / 27777 : 2389
7 / 27777 : 4010
8 / 27777 : error
9 / 27777 : 2815
10 / 27777 : 2929
11 / 27777 : 2389
12 / 27777 : 1245
13 / 27777 : 2960
14 / 27777 : 1950
15 / 27777 : 4327
16 / 27777 : 2084
17 / 27777 : 3163
18 / 27777 : 4129
19 / 27777 : 4536
20 / 27777 : 1200
21 / 27777 : 1484
22 / 27777 : 1578
23 / 27777 : 2126
24 / 27777 : 2494
25 / 27777 : 5133
26 / 27777 : 4357
27 / 27777 : 3531
28 / 27777 : 2505
29 / 27777 : 3692
30 / 27777 : 3692
31 / 27777 : 3650
32 / 27777 : 3452
33 / 27777 : 1819
34 / 27777 : 4342
35 / 27777 : 2984
36 / 27777 : 3627
37 / 27777 : 4641
38 / 27777 : 2377
39 / 27777 : 3498
40 / 27777 : 4802
41 / 27777 : 3661
42 / 27777 : 4140
43 / 27777 : 4907
44 / 27777 : 6679
45 / 27777 : 4574
46 / 27777 : 4190
47 / 27777 : 2860
48 / 27777 : 3684
49 / 27777 : 5336
50 / 27777 : 6525
51 / 27777 : 5071
52 / 27777 : 7556
53 / 27777 : 5158
54 / 27777 : 508
55 / 27777 : 3369
56

KeyboardInterrupt: 

In [15]:
df.to_json("0-10389_unique_jobs_with_content.json")

In [4]:
## fix errors
df_err = pd.read_json('0-10389_unique_jobs_with_content.json')

In [14]:
errors = []
for index, row in df_err[:10390].iterrows():
    print(index, "/", 10390)
    try:
        if row.content == "":
            wd.get(row.link)
            content = wd.find_element(
                By.CSS_SELECTOR, "div.show-more-less-html__markup")
            df_err.loc[index, 'content'] = content.get_attribute("innerText")
            print(index, "/", 10390,":",len(df_err.loc[index, 'content']))
            time.sleep(4)

    except:
        df_err.loc[index, 'content'] = ""
        errors.append(index)
        print(index, "/", 10390,": error")
        time.sleep(5)



0 / 10390
1 / 10390
2 / 10390
3 / 10390
4 / 10390
5 / 10390
6 / 10390
7 / 10390
8 / 10390
9 / 10390
10 / 10390
11 / 10390
12 / 10390
13 / 10390
14 / 10390
15 / 10390
16 / 10390
17 / 10390
18 / 10390
19 / 10390
20 / 10390
21 / 10390
22 / 10390
23 / 10390
24 / 10390
25 / 10390
26 / 10390
27 / 10390
28 / 10390
29 / 10390
30 / 10390
31 / 10390
32 / 10390
33 / 10390
34 / 10390
35 / 10390
36 / 10390
37 / 10390
38 / 10390
39 / 10390
40 / 10390
41 / 10390
42 / 10390
43 / 10390
44 / 10390
45 / 10390
46 / 10390
47 / 10390
48 / 10390
49 / 10390
50 / 10390
51 / 10390
52 / 10390
53 / 10390
54 / 10390
55 / 10390
56 / 10390
57 / 10390
58 / 10390
59 / 10390
60 / 10390
61 / 10390
62 / 10390
63 / 10390
64 / 10390
65 / 10390
66 / 10390
67 / 10390
68 / 10390
69 / 10390
70 / 10390
71 / 10390
72 / 10390
73 / 10390
74 / 10390
75 / 10390
76 / 10390
77 / 10390
78 / 10390
79 / 10390
80 / 10390
81 / 10390
82 / 10390
83 / 10390
84 / 10390
85 / 10390
86 / 10390
87 / 10390
88 / 10390
89 / 10390
90 / 10390
91 / 1039

In [15]:
print(len(errors)) 

0


In [16]:
df_err.to_json("10389_unique_jobs_with_content.json")


## partial content fetching

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
df = pd.read_json('0-10389_unique_jobs_with_content.json')
iFrom = 10390
iTo = 12500
name = str(iFrom) + "-" + str(iTo)+"_unique_jobs_with_content.json"
name

'10390-12500_unique_jobs_with_content.json'

In [3]:
options = webdriver.ChromeOptions() 
options.headless = True 
wd = webdriver.Chrome('./chromedriver.exe', options=options)

C:\Users\denni\AppData\Local\Temp/ipykernel_10552/1245260802.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('./chromedriver.exe', options=options)


In [4]:
errors = []
for index, row in df[iFrom:iTo].iterrows():
    # print(index, "/", iTo)
    try:
        if row.content == None:
            wd.get(row.link)
            content = wd.find_element(
                By.CSS_SELECTOR, "div.show-more-less-html__markup")
            df.loc[index, 'content'] = content.get_attribute("innerText")
            print(index, "/", iTo,":",len(df.loc[index, 'content']))
            time.sleep(5)

    except:
        df.loc[index, 'content'] = ""
        errors.append(index)
        print(index, "/", iTo,": error")
        time.sleep(10)
df.to_json(name)
df.to_json("current_content_fetching.json")

10390 / 12500 : 2561
10391 / 12500 : 2556
10392 / 12500 : 2184
10393 / 12500 : 1223
10394 / 12500 : 3935
10395 / 12500 : 3095
10396 / 12500 : 4177
10397 / 12500 : error
10398 / 12500 : 2768
10399 / 12500 : 4949
10400 / 12500 : 3580
10401 / 12500 : 2033
10402 / 12500 : 3148
10403 / 12500 : 1518
10404 / 12500 : 2085
10405 / 12500 : 2197
10406 / 12500 : 1239
10407 / 12500 : 2437
10408 / 12500 : 5834
10409 / 12500 : 2848
10410 / 12500 : 1655
10411 / 12500 : 2133
10412 / 12500 : 2985
10413 / 12500 : 2416
10414 / 12500 : 3800
10415 / 12500 : 2441
10416 / 12500 : 1502
10417 / 12500 : 1833
10418 / 12500 : 1916
10419 / 12500 : error
10420 / 12500 : 5052
10421 / 12500 : 2685
10422 / 12500 : 2589
10423 / 12500 : 2585
10424 / 12500 : 1242
10425 / 12500 : 2927
10426 / 12500 : 3638
10427 / 12500 : 3167
10428 / 12500 : 2515
10429 / 12500 : 2800
10430 / 12500 : 2433
10431 / 12500 : 1785
10432 / 12500 : 4366
10433 / 12500 : 2744
10434 / 12500 : 4102
10435 / 12500 : 2384
10436 / 12500 : 3249
10437 / 125

In [20]:
print(len(errors))

1


In [21]:
errors = []
for index, row in df[iFrom:iTo].iterrows():
    print(index, "/", iTo)
    try:
        if row.content == "" or row.content == None:
            wd.get(row.link)
            content = wd.find_element(
                By.CSS_SELECTOR, "div.show-more-less-html__markup")
            df.loc[index, 'content'] = content.get_attribute("innerText")
            print(index, "/", iTo,":",len(df.loc[index, 'content']))
            time.sleep(3)

    except:
        df.loc[index, 'content'] = ""
        errors.append(index)
        print(index, "/", iTo,": error")
        time.sleep(10)

10390 / 12500
10391 / 12500
10392 / 12500
10393 / 12500
10394 / 12500
10395 / 12500
10396 / 12500
10397 / 12500
10398 / 12500
10399 / 12500
10400 / 12500
10401 / 12500
10402 / 12500
10403 / 12500
10404 / 12500
10405 / 12500
10406 / 12500
10407 / 12500
10408 / 12500
10409 / 12500
10410 / 12500
10411 / 12500
10412 / 12500
10413 / 12500
10414 / 12500
10415 / 12500
10416 / 12500
10417 / 12500
10418 / 12500
10419 / 12500
10420 / 12500
10421 / 12500
10422 / 12500
10423 / 12500
10424 / 12500
10425 / 12500
10426 / 12500
10427 / 12500
10428 / 12500
10429 / 12500
10430 / 12500
10431 / 12500
10432 / 12500
10433 / 12500
10434 / 12500
10435 / 12500
10436 / 12500
10437 / 12500
10438 / 12500
10439 / 12500
10440 / 12500
10441 / 12500
10442 / 12500
10443 / 12500
10444 / 12500
10445 / 12500
10446 / 12500
10447 / 12500
10448 / 12500
10449 / 12500
10450 / 12500
10451 / 12500
10452 / 12500
10453 / 12500
10454 / 12500
10455 / 12500
10456 / 12500
10457 / 12500
10458 / 12500
10459 / 12500
10460 / 12500
10461 

In [22]:
df.to_json("current_content_fetching.json")

In [23]:
print(len(errors))
print(df.shape)

0
(27777, 7)


In [60]:
df.to_json("current_content_fetching.json")

# final data

In [61]:
df_final = pd.read_json('current_content_fetching.json')
df_final_noerr = pd.read_json("10389_unique_jobs_with_content.json")

In [62]:
print(len(df_final[df_final["content"]=='']))
print(len(df_final_noerr[df_final_noerr["content"]=='']))

805
0


In [55]:
df_final[df_final["content"]=='']

,link,title,company,location,listdate,city,content
8,https://de.linkedin.com/jobs/view/machine-lear...,Machine Learning Engineer (f/m/x),nyris GmbH,Berlin,2023-01-25,Berlin,
1334,https://de.linkedin.com/jobs/view/research-sci...,Research Scientist (f/m/d) Target Identificati...,Evotec,Hamburg,2023-01-24,Hamburg,
2377,https://de.linkedin.com/jobs/view/devops-engin...,DevOps Engineer (m/w/d),berg-it Projektdienstleistungen GmbH,Nürnberg,2023-01-05,München,
4294,https://de.linkedin.com/jobs/view/android-entw...,Android Entwickler (m/w/d) InsurTech,CHECK24 Vergleichsportal,Frankfurt,2023-01-23,Frankfurt am Main,
4548,https://de.linkedin.com/jobs/view/junior-java-...,Junior Java Developer (m/f/d) e*star,Exxeta,Frankfurt,2023-01-21,Frankfurt am Main,
...,...,...,...,...,...,...,...
10385,https://de.linkedin.com/jobs/view/softwareentw...,Softwareentwickler - Frontend| E-Commerce (m/w...,digital.manufaktur GmbH,Hannover,2022-11-30,Hannover,
10386,https://de.linkedin.com/jobs/view/entwickler-p...,Entwickler / Programmierer Mikrokontroller / E...,Brunel,Hannover,2022-12-27,Hannover,
10387,https://de.linkedin.com/jobs/view/devops-engin...,DevOps Engineer - Microservices / Testing / Ku...,iS Software GmbH,Hannover,2022-12-24,Hannover,
10388,https://de.linkedin.com/jobs/view/softwareentw...,Softwareentwickler (m/w/d),inTime Express Logistik GmbH,Hannover,2022-10-05,Hannover,


In [56]:
iFrom

10390

In [65]:
df_final[:iFrom] = df_final_noerr[:iFrom]

In [66]:
print(df_final.shape)
df_final[df_final["content"]=='']

(27777, 7)


,link,title,company,location,listdate,city,content


In [70]:
print(df_final.iloc[10389]["link"])
print(df_final.iloc[10389]["content"])


https://de.linkedin.com/jobs/view/junior-sap-entwickler-at-brunel-3436809459
Mit nur einer Bewerbung haben Sie über uns Zugang zu hunderten, offenen Positionen. Nach Ihrer Bewerbung treten wir mit Ihnen in Kontakt, um Ihre Wünsche und Anforderungen bei unserem weiterem Handeln in den Mittelpunkt stellen zu können. Denn unser erster Schritt ist, Ihre Interessen mit den offenen Anfragen unserer Kunden abzugleichen. Danach stellen wir Ihnen verschiedene Möglichkeiten vor und entscheiden gemeinsam mit Ihnen, wo Ihre weitere Reise hingehen soll. Dabei sind Ihrer Vorstellung keine Grenzen gesetzt. Über unser Netzwerk können wir Ihnen branchenunabhängig die gesamte Vielfalt des Engineerings und IT bieten, sowie die Eintrittskarte in einen mittelständischen Hidden Champion oder Global Player. Neben einer optimalen Vorbereitung auf das Vorstellungsgespräch bei unserem Kunden, stehen wir Ihnen bei allen offenen Fragen stärkend zur Seite.

Ihre Aufgabe


Die Durchführung spannender Entwicklungspr

In [71]:
df_final.to_json("current_content_fetching.json")

## final set test

In [76]:
df_t = pd.read_json('current_content_fetching.json')
df_t[df_t["content"]=='']

,link,title,company,location,listdate,city,content


In [77]:
len(df_t[df_t["content"].isna()])

0

In [78]:
df_tu = df_t.drop_duplicates(subset=['link'])
print(len(df_tu))

27777
